In [1]:
import numpy as np
import json
import os
from pathlib import Path
from typing import List, Dict, Tuple
import mido

## 1. Load EigenSpace Data

In [2]:
def load_dissonance_field(
    dataset_path: str = "../dataset/EigenSpace_Data",
    base_freq: int = 220,
    n_points: int = 150
) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """Load pre-computed 150³ dissonance field from binary chunks."""
    chunk_files = sorted([
        f for f in os.listdir(dataset_path)
        if f.startswith(f"harmonic-{base_freq}Hz-{n_points}nodes-chunk")
    ])
    
    if not chunk_files:
        raise FileNotFoundError(f"No chunks found for {base_freq}Hz, {n_points} nodes")
    
    all_data = []
    for chunk_file in chunk_files:
        chunk_path = os.path.join(dataset_path, chunk_file)
        all_data.append(np.fromfile(chunk_path, dtype=np.float32))
    
    dissonance_3d = np.concatenate(all_data).reshape((n_points, n_points, n_points))
    coord_range = np.linspace(1.0, 2.0, n_points)
    
    print(f"✓ Loaded {n_points}³ dissonance field")
    print(f"  Range: {dissonance_3d.min():.2f} to {dissonance_3d.max():.2f}")
    
    return coord_range, coord_range.copy(), coord_range.copy(), dissonance_3d


alpha_range, beta_range, gamma_range, dissonance_3d = load_dissonance_field()

✓ Loaded 150³ dissonance field
  Range: 0.67 to 30.62


## 2. Load 53-TET Intervals & Build Chord Library

In [3]:
# Core functions
def midi_to_frequency(midi_note: float) -> float:
    return 440.0 * (2.0 ** ((midi_note - 69.0) / 12.0))

def frequency_to_midi(freq: float) -> float:
    return 69.0 + 12.0 * np.log2(freq / 440.0)

def get_dissonance(alpha: float, beta: float, gamma: float,
                   alpha_range: np.ndarray, beta_range: np.ndarray,
                   gamma_range: np.ndarray, dissonance_3d: np.ndarray) -> float:
    """Trilinear interpolation into the dissonance field."""
    def find_idx(val, arr):
        idx = np.searchsorted(arr, val) - 1
        return max(0, min(idx, len(arr) - 2))
    
    ia, ib, ig = find_idx(alpha, alpha_range), find_idx(beta, beta_range), find_idx(gamma, gamma_range)
    
    def frac(val, arr, idx):
        return (val - arr[idx]) / (arr[idx + 1] - arr[idx]) if arr[idx + 1] != arr[idx] else 0
    
    fa, fb, fg = frac(alpha, alpha_range, ia), frac(beta, beta_range, ib), frac(gamma, gamma_range, ig)
    
    c000 = dissonance_3d[ia, ib, ig]
    c001 = dissonance_3d[ia, ib, ig + 1]
    c010 = dissonance_3d[ia, ib + 1, ig]
    c011 = dissonance_3d[ia, ib + 1, ig + 1]
    c100 = dissonance_3d[ia + 1, ib, ig]
    c101 = dissonance_3d[ia + 1, ib, ig + 1]
    c110 = dissonance_3d[ia + 1, ib + 1, ig]
    c111 = dissonance_3d[ia + 1, ib + 1, ig + 1]
    
    c00 = c000 * (1 - fa) + c100 * fa
    c01 = c001 * (1 - fa) + c101 * fa
    c10 = c010 * (1 - fa) + c110 * fa
    c11 = c011 * (1 - fa) + c111 * fa
    
    c0 = c00 * (1 - fb) + c10 * fb
    c1 = c01 * (1 - fb) + c11 * fb
    
    return c0 * (1 - fg) + c1 * fg

In [4]:
# Load 53-TET intervals
TET53_FILE = Path('../dataset/EigenSpace_Data/53tet_intervals.json')
with open(TET53_FILE, 'r') as f:
    tet53_intervals = json.load(f)

print(f"53-TET Intervals:")
print(f"  Thirds: {len(tet53_intervals['thirds'])}")
print(f"  Fifths: {len(tet53_intervals['fifths'])}")
print(f"  Sevenths: {len(tet53_intervals['sevenths'])}")

53-TET Intervals:
  Thirds: 10
  Fifths: 7
  Sevenths: 10


In [5]:
# Build 53-TET triads (70 combinations)
tet53_triads = []
for third_name, third_data in tet53_intervals['thirds'].items():
    for fifth_name, fifth_data in tet53_intervals['fifths'].items():
        beta = third_data['ratio']
        gamma = fifth_data['ratio']
        if 1.0 <= beta <= 2.0 and 1.0 <= gamma <= 2.0 and beta <= gamma:
            diss = get_dissonance(1.0, beta, gamma, alpha_range, beta_range, gamma_range, dissonance_3d)
            tet53_triads.append({
                'name': f"{third_name}-{fifth_name}",
                'beta': beta, 'gamma': gamma,
                'dissonance': float(diss)
            })

tet53_triads.sort(key=lambda x: x['dissonance'])
print(f"\n✓ Built {len(tet53_triads)} 53-TET triads")
print(f"  Best: {tet53_triads[0]['name']} (D={tet53_triads[0]['dissonance']:.2f})")


✓ Built 70 53-TET triads
  Best: vM-P (D=11.24)


In [6]:
# Build 53-TET tetrachords (700 combinations)
tet53_tetrachords = []
for third_name, third_data in tet53_intervals['thirds'].items():
    for fifth_name, fifth_data in tet53_intervals['fifths'].items():
        for seventh_name, seventh_data in tet53_intervals['sevenths'].items():
            alpha = third_data['ratio']
            beta = fifth_data['ratio']
            gamma = seventh_data['ratio']
            if 1.0 <= alpha <= 2.0 and 1.0 <= beta <= 2.0 and 1.0 <= gamma <= 2.0:
                if alpha <= beta <= gamma:
                    diss = get_dissonance(alpha, beta, gamma, alpha_range, beta_range, gamma_range, dissonance_3d)
                    tet53_tetrachords.append({
                        'name': f"{third_name}-{fifth_name}-{seventh_name}",
                        'alpha': alpha, 'beta': beta, 'gamma': gamma,
                        'dissonance': float(diss)
                    })

tet53_tetrachords.sort(key=lambda x: x['dissonance'])
print(f"✓ Built {len(tet53_tetrachords)} 53-TET tetrachords")
print(f"  Best: {tet53_tetrachords[0]['name']} (D={tet53_tetrachords[0]['dissonance']:.2f})")

✓ Built 700 53-TET tetrachords
  Best: ^m-P-^m (D=12.40)


## 3. 53-TET MPE MIDI Generator

In [7]:
def compute_53tet_corrections(
    chord_notes: List[int],
    tet53_triads: List[Dict],
    tet53_tetrachords: List[Dict],
    alpha_range: np.ndarray,
    beta_range: np.ndarray,
    gamma_range: np.ndarray,
    dissonance_3d: np.ndarray,
    max_cents: float = 50.0
) -> Dict[int, float]:
    """
    Compute 53-TET pitch corrections for a chord.
    Returns: {midi_note: correction_cents} capped to ±max_cents.
    """
    n = len(chord_notes)
    if n < 3:
        return {}
    
    sorted_notes = sorted(chord_notes)
    freqs = [midi_to_frequency(note) for note in sorted_notes]
    root = freqs[0]
    ratios = [f / root for f in freqs]
    
    corrections = {}
    max_ratio_factor = 2 ** (max_cents / 1200)
    
    if n == 3:
        beta, gamma = ratios[1], ratios[2]
        if not (1.0 <= beta <= 2.0 and 1.0 <= gamma <= 2.0 and beta <= gamma):
            return {}
        
        orig_diss = get_dissonance(1.0, beta, gamma, alpha_range, beta_range, gamma_range, dissonance_3d)
        best = None
        
        for t in tet53_triads:
            if ((1/max_ratio_factor) <= (t['beta']/beta) <= max_ratio_factor and
                (1/max_ratio_factor) <= (t['gamma']/gamma) <= max_ratio_factor and
                t['dissonance'] < orig_diss):
                if best is None or t['dissonance'] < best['dissonance']:
                    best = t
        
        if best:
            corrections[sorted_notes[0]] = 0.0
            for i, key in enumerate(['beta', 'gamma'], 1):
                target_midi = frequency_to_midi(root * best[key])
                corr = (target_midi - sorted_notes[i]) * 100
                corrections[sorted_notes[i]] = max(-max_cents, min(max_cents, corr))
                
    elif n == 4:
        alpha, beta, gamma = ratios[1], ratios[2], ratios[3]
        if not (1.0 <= alpha <= beta <= gamma <= 2.0):
            return {}
        
        orig_diss = get_dissonance(alpha, beta, gamma, alpha_range, beta_range, gamma_range, dissonance_3d)
        best = None
        
        for t in tet53_tetrachords:
            if ((1/max_ratio_factor) <= (t['alpha']/alpha) <= max_ratio_factor and
                (1/max_ratio_factor) <= (t['beta']/beta) <= max_ratio_factor and
                (1/max_ratio_factor) <= (t['gamma']/gamma) <= max_ratio_factor and
                t['dissonance'] < orig_diss):
                if best is None or t['dissonance'] < best['dissonance']:
                    best = t
        
        if best:
            corrections[sorted_notes[0]] = 0.0
            for i, key in enumerate(['alpha', 'beta', 'gamma'], 1):
                target_midi = frequency_to_midi(root * best[key])
                corr = (target_midi - sorted_notes[i]) * 100
                corrections[sorted_notes[i]] = max(-max_cents, min(max_cents, corr))
    
    return corrections

In [8]:
def generate_53tet_mpe(
    input_path: str,
    output_path: str,
    tet53_triads: List[Dict],
    tet53_tetrachords: List[Dict],
    alpha_range: np.ndarray,
    beta_range: np.ndarray,
    gamma_range: np.ndarray,
    dissonance_3d: np.ndarray,
    max_cents: float = 50.0,
    pitch_bend_range: int = 48
) -> Dict:
    """
    Generate 53-TET enhanced MPE MIDI from input file.
    
    Args:
        input_path: Source MIDI file
        output_path: Output file path
        max_cents: Maximum correction (default ±50¢)
        pitch_bend_range: MPE pitch bend range in semitones (default 48)
    
    Returns:
        Statistics dict
    """
    original = mido.MidiFile(input_path)
    new_mid = mido.MidiFile(ticks_per_beat=original.ticks_per_beat, type=original.type)
    
    # Copy tempo track
    new_mid.tracks.append(original.tracks[0].copy())
    
    # Get note track
    note_track = original.tracks[1] if len(original.tracks) > 1 else original.tracks[0]
    
    # Parse all events with absolute timing
    abs_time = 0
    events = []
    for msg in note_track:
        abs_time += msg.time
        events.append({'time': abs_time, 'msg': msg})
    
    # Find chords (notes starting at same time)
    chord_starts = {}
    for e in events:
        msg = e['msg']
        if msg.type == 'note_on' and msg.velocity > 0:
            t = e['time']
            if t not in chord_starts:
                chord_starts[t] = []
            chord_starts[t].append((msg.note, msg.channel))
    
    # Compute corrections
    all_corrections = {}
    stats = {'chords': len(chord_starts), 'corrected': 0, 'notes': 0}
    
    for time, notes_info in chord_starts.items():
        if len(notes_info) >= 3:
            chord_notes = [n[0] for n in notes_info]
            corrections = compute_53tet_corrections(
                chord_notes, tet53_triads, tet53_tetrachords,
                alpha_range, beta_range, gamma_range, dissonance_3d, max_cents
            )
            if corrections:
                stats['corrected'] += 1
                for note, cents in corrections.items():
                    if abs(cents) > 0.1:
                        all_corrections[(time, note)] = cents
                        stats['notes'] += 1
    
    # Build output track with pitch bends
    new_track = mido.MidiTrack()
    output_events = []
    active_bends = {}
    
    for e in events:
        msg = e['msg']
        t = e['time']
        
        if msg.type == 'note_on' and msg.velocity > 0:
            cents = all_corrections.get((t, msg.note), 0.0)
            if abs(cents) > 0.1:
                pb_value = int((cents / 100.0) / pitch_bend_range * 8192)
                pb_value = max(-8192, min(8191, pb_value))
                output_events.append({
                    'time': t, 'order': 0,
                    'msg': mido.Message('pitchwheel', channel=msg.channel, pitch=pb_value)
                })
                active_bends[msg.note] = msg.channel
            output_events.append({'time': t, 'order': 1, 'msg': msg})
            
        elif msg.type == 'note_off' or (msg.type == 'note_on' and msg.velocity == 0):
            output_events.append({'time': t, 'order': 2, 'msg': msg})
            if msg.note in active_bends:
                output_events.append({
                    'time': t, 'order': 3,
                    'msg': mido.Message('pitchwheel', channel=active_bends[msg.note], pitch=0)
                })
                del active_bends[msg.note]
        else:
            output_events.append({'time': t, 'order': 5 if msg.is_meta else 1, 'msg': msg})
    
    # Sort and convert to delta time
    output_events.sort(key=lambda x: (x['time'], x['order']))
    prev_time = 0
    for e in output_events:
        e['msg'].time = e['time'] - prev_time
        new_track.append(e['msg'])
        prev_time = e['time']
    
    new_mid.tracks.append(new_track)
    new_mid.save(output_path)
    
    stats['duration'] = new_mid.length
    return stats

## 4. Generate Output

In [9]:
# Configuration
INPUT_FILE = Path('../dataset/midi_files/mpe/47832_Something_C_major.mid')
OUTPUT_DIR = Path('../dataset/midi_files/microtonal')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

OUTPUT_FILE = OUTPUT_DIR / f"{INPUT_FILE.stem}_53TET.mid"

print("═" * 60)
print("  53-TET ENHANCED MIDI GENERATOR")
print("═" * 60)
print(f"\nInput:  {INPUT_FILE}")
print(f"Output: {OUTPUT_FILE}")
print(f"Max correction: ±50 cents")
print()

# Generate
stats = generate_53tet_mpe(
    str(INPUT_FILE),
    str(OUTPUT_FILE),
    tet53_triads,
    tet53_tetrachords,
    alpha_range, beta_range, gamma_range,
    dissonance_3d,
    max_cents=50.0
)

print(f"✅ Generated: {OUTPUT_FILE.name}")
print(f"   Duration: {stats['duration']:.1f} seconds")
print(f"   Chords: {stats['corrected']}/{stats['chords']} corrected")
print(f"   Notes with pitch bend: {stats['notes']}")

════════════════════════════════════════════════════════════
  53-TET ENHANCED MIDI GENERATOR
════════════════════════════════════════════════════════════

Input:  ../dataset/midi_files/mpe/47832_Something_C_major.mid
Output: ../dataset/midi_files/microtonal/47832_Something_C_major_53TET.mid
Max correction: ±50 cents

✅ Generated: 47832_Something_C_major_53TET.mid
   Duration: 372.0 seconds
   Chords: 81/165 corrected
   Notes with pitch bend: 180


In [10]:
# Verify output
print("\n═══ OUTPUT VERIFICATION ═══\n")

mid = mido.MidiFile(str(OUTPUT_FILE))

# Collect pitch bend stats
pitch_bends = []
for track in mid.tracks:
    for msg in track:
        if msg.type == 'pitchwheel' and msg.pitch != 0:
            cents = msg.pitch / 8192 * 48 * 100
            pitch_bends.append(cents)

print(f"Pitch bend corrections: {len(pitch_bends)}")
if pitch_bends:
    print(f"Range: {min(pitch_bends):.1f}¢ to {max(pitch_bends):.1f}¢")
    print(f"Mean: {np.mean(pitch_bends):.1f}¢")

print(f"\n🎵 Open in DAW: {OUTPUT_FILE}")
print("   You should see pitch bend automation on each channel.")


═══ OUTPUT VERIFICATION ═══

Pitch bend corrections: 180
Range: -14.6¢ to 18.8¢
Mean: -3.9¢

🎵 Open in DAW: ../dataset/midi_files/microtonal/47832_Something_C_major_53TET.mid
   You should see pitch bend automation on each channel.
